In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/ner-dataset/NER_Dataset.csv')

In [2]:
df

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."
...,...,...,...,...
47954,Sentence: 9995,"['Opposition', 'leader', 'Mir', 'Hossein', 'Mo...","['NNP', 'NN', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBN...","['O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O..."
47955,Sentence: 9996,"['On', 'Thursday', ',', 'Iranian', 'state', 'm...","['IN', 'NNP', ',', 'JJ', 'NN', 'NNS', 'VBN', '...","['O', 'B-tim', 'O', 'B-gpe', 'O', 'O', 'O', 'O..."
47956,Sentence: 9997,"['Following', 'Iran', ""'s"", 'disputed', 'June'...","['VBG', 'NNP', 'POS', 'JJ', 'NNP', 'CD', 'NNS'...","['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O'..."
47957,Sentence: 9998,"['Since', 'then', ',', 'authorities', 'have', ...","['IN', 'RB', ',', 'NNS', 'VBP', 'VBN', 'JJ', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


# EDA
Investigate how many Tags are in the dataset

In [3]:
import ast 

# Current data has list form read in literal. Change this to list form
for i, row in df.iterrows():
    df.loc[i, 'Word'] = ast.literal_eval(df.loc[i, 'Word'])
    df.loc[i, 'Tag'] = ast.literal_eval(df.loc[i, 'Tag'])

unique_tags = set()

def add_unique_tags(row):
    taglist = row['Tag']
    for tag in taglist:
        # Add the tag only if not in the unique_tags set
        if tag not in unique_tags:
            unique_tags.add(tag)

df.apply(add_unique_tags, axis=1)
unique_tags

{'B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O'}

In [4]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Word'], df['Tag'], test_size=0.2, random_state=42)

In [5]:
X_train[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']